Convert the Physics Derivation Graph's v7 JSON format to Neo4j Cypher

To load the data into ui_v8,

```bash
docker exec `docker ps | grep community | cut -d' ' -f1` bin/cypher-shell --file dumping_grounds/symbols.cypher 
docker exec `docker ps | grep community | cut -d' ' -f1` bin/cypher-shell --file dumping_grounds/infrules.cypher 
```

In [1]:
import json
#import pandas
import time
import random

# data

In [2]:
# from v7_pickle_web_interface/flask/
with open('data.json','r') as file_handle:
    data = json.loads(file_handle.read())

In [3]:
data.keys()

dict_keys(['derivations', 'expr local to global', 'expressions', 'inference rules', 'measures', 'operators', 'symbols', 'units'])


`expr local to global` is made irrelevant by the use of a property graph

# symbols

In [4]:
str_to_prnt = ""
for symbol_ID, symbol_dict in data['symbols'].items():
    ref_str = ""
    if 'references' in symbol_dict.keys():
        for this_ref in symbol_dict['references']:
            ref_str += this_ref + " and "
        ref_str = ref_str[:-5]

    
    str_to_prnt += ":begin"+"\n"
    str_to_prnt += 'UNWIND [{id:"000'+symbol_ID+'",\n'
    str_to_prnt += '         properties:{reference_latex:"'+ref_str+'",\n'
    str_to_prnt += '                     dimension_time: '+str(symbol_dict['dimensions']['time'])+',\n'
    str_to_prnt += '                     dimension_electric_charge:'+str(symbol_dict['dimensions']['electric charge'])+',\n'
    str_to_prnt += '                     dimension_luminous_intensity:'+str(symbol_dict['dimensions']['luminous intensity'])+',\n'
    str_to_prnt += '                     dimension_length:'+str(symbol_dict['dimensions']['length'])+',\n'
    str_to_prnt += '                     dimension_amount_of_substance:'+str(symbol_dict['dimensions']['amount of substance'])+',\n'
    str_to_prnt += '                     dimension_mass:'+str(symbol_dict['dimensions']['mass'])+',\n'
    str_to_prnt += '                     dimension_temperature:'+str(symbol_dict['dimensions']['temperature'])+',\n'
    str_to_prnt += '                     description_latex:"",\n'
    str_to_prnt += '                     latex:"'+symbol_dict['latex']+'",\n'
    if 'name' in symbol_dict.keys():
        str_to_prnt += '                     name_latex:"'+symbol_dict['name']+'",\n'
    else:
        str_to_prnt += '                     name_latex:"",\n'
    str_to_prnt += '                     variable_or_constant:"'+symbol_dict['category']+'",\n'
    str_to_prnt += '                     author_name_latex:"ben",\n'
    if type(symbol_dict['scope'])==type([]):
        str_to_prnt += '                     scope:"'+symbol_dict['scope'][0]+'",\n'
    else:
        str_to_prnt += '                     scope:"'+symbol_dict['scope']+'",\n'
    str_to_prnt += '                     domain:"any"}}] AS row'+'\n'
    str_to_prnt += 'CREATE (n:scalar{id: row.id}) SET n += row.properties SET n:symbol;'+'\n'
    str_to_prnt += ':commit'+'\n'
    str_to_prnt += "CALL db.awaitIndexes(300);"+'\n'

    if symbol_dict['category']=="constant":
        print("TODO: "+symbol_dict['latex']+" has value list "+str(symbol_dict['values'])+"\n")
    
with open("symbols.cypher","w") as file_handle:
    file_handle.write(str_to_prnt)


TODO: \hbar has value list [{'units': 'meter^2 kilogram second^-1', 'value': '1.0545718*10^{-34}'}]

TODO: k_{Boltzmann} has value list [{'units': 'meter^2 kilogram second^-2 Kelvin^-1', 'value': '1.38064852 10^{-23}'}]

TODO: \alpha has value list [{'units': 'dimensionless', 'value': '1/137.03599999'}]

TODO: e has value list [{'units': 'Columb', 'value': '1.602*10^{-19}'}]

TODO: m_e has value list [{'units': 'kg', 'value': '9.1093837015E^{-31}'}]

TODO: \exp has value list [{'units': 'unitless', 'value': '2.71828'}]

TODO: r_{\rm Bohr} has value list [{'units': 'm', 'value': '5.29E^{-11}'}]

TODO: \pi has value list [{'units': 'dimensionless', 'value': '3.1415'}]

TODO: r_{\rm Earth} has value list [{'units': 'm', 'value': '6.3781*10^{6}'}]

TODO: c has value list [{'units': 'meters/second', 'value': '299792458'}]

TODO: v_u has value list [{'units': 'm/s', 'value': '36100'}]

TODO: m_{\rm Earth} has value list [{'units': 'kg', 'value': '5.97237*10^24'}]

TODO: m_p has value list [{

# derivations

# units

# symbols

# operators

# inference rules

In [5]:
str_to_prnt = ""
list_of_IDs = random.sample(list(range(100, 1000)),len(data['inference rules']))

index = -1
for infrule_name, infrule_dict in data['inference rules'].items():
    index+=1
    str_to_prnt +=':begin'+'\n'
    str_to_prnt +='UNWIND [{id:"111'+str(list_of_IDs[index])+'",'+'\n'
    str_to_prnt +='         properties:{name_latex:"'+infrule_name+'",'+'\n'
    str_to_prnt +='                     number_of_inputs:'+str(infrule_dict['number of inputs'])+','+'\n'
    str_to_prnt +='                     number_of_feeds:'+str(infrule_dict['number of feeds'])+','+'\n'
    str_to_prnt +='                     number_of_outputs:'+str(infrule_dict['number of outputs'])+','+'\n'
    str_to_prnt +='                     author_name_latex:"ben",'+'\n'
    if len(infrule_dict['notes'])>1:
        str_to_prnt +='                     notes_latex:"'+infrule_dict['notes'].replace('"',"'")+'",'+'\n'
    else:
        str_to_prnt +='                     notes_latex:"",'+'\n'
    if len(infrule_dict['assumptions'])>1:
        str_to_prnt +='                     assumptions_latex:"'+infrule_dict['assumptions']+'",'+'\n'
    else:
        str_to_prnt +='                     assumptions_latex:"",'+'\n'
    str_to_prnt +='                     latex:"'+infrule_dict['latex']+'"}}] AS row'+'\n'
    str_to_prnt +='CREATE (n:inference_rule{id: row.id}) SET n += row.properties;'+'\n'
    str_to_prnt +=':commit'+'\n'
    str_to_prnt += "CALL db.awaitIndexes(300);"+'\n'
    
with open("infrules.cypher","w") as file_handle:
    file_handle.write(str_to_prnt)


# measures